# [실습3] 엑셀 데이터와 PDF 문서를 모두 활용하는 챗봇

## 실습 목표
---
이전 실습에서 구현했던 다양한 기능을 한데 모아 하나의 챗봇으로 구성해 봅시다
- 엑셀 데이터로 데이터 분석 코드를 실행하고 답변하는 기능
- 엑셀 데이터로 데이터 분석 그래프를 그리고 저장하는 기능
- 시장 조사 문서 기반 QA 챗봇

## 실습 목차
---

1. **각 기능 별 그래프 노드 정의:** 추가하고자 하는 다양한 기능에 대응하는 노드를 정의합니다.

2. **챗봇 고도화:** LangGraph를 활용해서 엑셀 데이터와 시장 조사 문서를 모두 활용할 수 있는 챗봇을 구성합니다.

## 실습 개요
---
엑셀 데이터와 시장 조사 문서를 모두 활용할 수 있는 챗봇을 구성하고 사용해봅니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [ ]:
import contextlib
import io
import os

import pandas as pd
from IPython.display import Image, display
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langgraph.graph import END, StateGraph
from typing_extensions import TypedDict

- Ollama를 통해 Mistral 7B 모델을 불러옵니다.

In [ ]:
!ollama pull mistral:7b

mistral:7b 모델을 사용하는 ChatOllama 객체를 생성하고, 엑셀 데이터를 불러옵니다.

In [ ]:
llm = ChatOllama(model="mistral:7b")
route_llm = ChatOllama(model="mistral:7b", format="json")

# 데이터를 불러오고, 이름과 컬럼명을 저장합니다.
data_dir = './data'
df_inkjet = pd.read_csv(os.path.join(data_dir, 'InkjetDB_preprocessing.csv'), index_col=0)

# 데이터를 저장한 변수명을 LLM에 제공하여 이 변수를 활용하는 코드를 작성하게 할 수 있습니다.
df_name = "df_inkjet"
df_columns = ", ".join(df_inkjet.columns)

시장 조사 문건을 불러와서 `OllamaEmbeddings`를 활용해 벡터로 변환하고, FAISS DB를 활용하여 저장합니다.
- 출처: 한국소비자원의 2022년 키오스크(무인정보단말기) 이용 실태조사 보고서
  - https://www.kca.go.kr/smartconsumer/sub.do?menukey=7301&mode=view&no=1003409523&page=2&cate=00000057

- 벡터 변환 및 저장 과정은 약 3분 정도 소요됩니다.

In [ ]:
%%time
embeddings = OllamaEmbeddings(model="mistral:7b")

# 시장 조사 문건을 불러옵니다.
doc_path = os.path.join(data_dir, '키오스크(무인정보단말기) 이용실태 조사.pdf')
loader = PyPDFLoader(doc_path)
docs = loader.load()

vectorstore = FAISS.from_documents(
    docs,
    embedding=embeddings
)

db_retriever = vectorstore.as_retriever()

## 1. 각 기능 별 그래프 노드 정의
- 추가하고자 하는 다양한 기능에 대응하는 노드를 정의합니다.

이전 실습에서 사용한 노드와 관련 함수를 먼저 정의합니다.

In [ ]:
# LLM이 생성한 코드를 파싱하는 함수를 정의합니다.
def python_code_parser(input: str) -> str:
    # LLM은 대부분 ``` 블럭 안에 코드를 출력합니다. 이를 활용합니다.
    # ```python (코드) ```, 혹은 ``` (코드) ``` 형태로 출력됩니다. 두 경우 모두에 대응하도록 코드를 작성합니다.
    processed_input = input.replace("```python", "```").strip()
    parsed_input_list = processed_input.split("```")

    # 만약 ``` 블럭이 없다면, 입력 텍스트 전체가 코드라고 간주합니다.
    # 아닐 경우 이어지는 코드 실행 과정에서 예외 처리를 통해 오류를 확인할 수 있습니다.
    if len(parsed_input_list) == 1:
        return processed_input

    # 코드 부분만 추출합니다. 
    # LLM은 여러 코드 블럭에 걸쳐 필요한 코드를 출력할 수 있으므로, 코드가 있는 홀수 번째 텍스트를 모두 저장합니다.
    parsed_code_list = []
    for i in range(1, len(parsed_input_list), 2):
        parsed_code_list.append(parsed_input_list[i])
    
    # 코드 부분을 하나로 합칩니다.
    return "\n".join(parsed_code_list)

# 생성한 코드를 실행하는 함수를 정의합니다.
def run_code(input_code: str):
    # 코드가 출력한 값을 캡쳐하기 위한 StringIO 객체를 생성합니다.
    output = io.StringIO()
    try:
        # Redirect stdout to the StringIO object
        with contextlib.redirect_stdout(output):
            # Python 3.10 버전이므로, 키워드 인자를 사용할 수 없습니다.
            # 코드가 실행하면서 출력한 모든 결과를 캡쳐합니다.
            exec(input_code, {"df_inkjet": df_inkjet})
    except Exception as e:
        # 에러가 발생할 경우, 이를 StringIO 객체에 저장합니다.
        print(f"Error: {e}", file=output)
    # StringIO 객체에 저장된 값을 반환합니다.
    return output.getvalue()

In [ ]:
class State(TypedDict):
    # 그래프 상태의 속성을 정의합니다.
    # 질문, LLM이 생성한 텍스트, 데이터, 코드를 저장합니다.
    question: str
    generation: str
    data: str
    code: str
        
# 그래프를 구성하기 위해 StateGraph 객체를 생성합니다.
# 생성자의 인자로 State를 전달하여 Node 간에 정보를 전달할 때 State type을 사용함을 명시합니다.
workflow = StateGraph(State)

그래프의 Node는 체인의 각 구성 요소에 대응합니다. Agent, Tool, LLM 등 그래프의 각 구성 요소를 의미합니다.

Edge는 Node를 연결하는 요소로, Node에서 정보를 어느 Node로 전달해야 하는지를 나타냅니다.
- 체인은 일렬로 이어져 있기 때문에, 사용자의 입력을 연결된 순서대로 통과시키면 원하는 결과를 얻을 수 있습니다.
- 하지만, 그래프는 일렬로 이어져 있지 않기 때문에 Edge를 통해 정보를 전달하는 순서 및 방향을 정해줘야 합니다.

In [ ]:
## Node 생성
# Node는 그래프에서 실행될 수 있는 작업을 정의합니다.
# Node는 함수로 정의되며, StateGraph를 정의할 때 사용한 State type을 입력으로 받습니다.
# Node는 state를 업데이트하거나, 새로운 state를 반환할 수 있습니다.

def query(state: State):
    """
    데이터를 쿼리하는 코드를 생성하고, 실행하고, 그 결과를 포함한 State를 반환합니다.
    위 과정은 앞서 정의한 `find_data` 함수를 활용합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        state (dict): 쿼리한 데이터를 포함한 새로운 State
    """
    print("---데이터 쿼리---") # 현재 상태를 확인하기 위한 Print문
    question = state["question"]

    # Retrieval
    # 이전 실습에서 `find_data` 함수를 사용했지만, 여기서는 query 함수에 해당 로직을 포함시켰습니다.
    system_message = "당신은 주어진 데이터를 분석하는 데이터 분석가입니다.\n"
    system_message += f"주어진 DataFrame에서 데이터를 출력하여 주어진 질문에 답할 수 있는 파이썬 코드를 작성하세요. "
    system_message += f"{df_name} DataFrame에 액세스할 수 있습니다.\n"
    system_message += f"`{df_name}` DataFrame에는 다음과 같은 열이 있습니다: {df_columns}\n"
    system_message += "데이터는 이미 로드되어 있으므로 데이터 로드 코드를 생략해야 합니다."

    message_with_data_info = [
        ("system", system_message),
        ("human", "{question}"),
    ]

    prompt_with_data_info = ChatPromptTemplate.from_messages(message_with_data_info)

    # 체인을 구성합니다.
    code_generate_chain = (
        {"question": RunnablePassthrough()}
        | prompt_with_data_info
        | llm 
        | StrOutputParser()
        | python_code_parser
    )
    code = code_generate_chain.invoke(question)
    data = run_code(code)
    return {"question": question, "code": code, "data": data, "generation": code}

def answer_with_data(state: State):
    """
    쿼리한 데이터를 바탕으로 답변을 생성합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        state (dict): LLM의 답변을 포함한 새로운 State
    """
    print("---데이터 기반 답변 생성---") # 현재 상태를 확인하기 위한 Print문
    question = state["question"]
    data = state["data"]

    # 데이터를 바탕으로 질문에 대답하는 코드를 생성합니다.
    reasoning_system_message = "당신은 데이터를 바탕으로 질문에 답하는 데이터 분석가입니다.\n"
    reasoning_system_message += f"사용자가 입력한 데이터를 바탕으로, 질문에 대답하세요."

    reasoning_user_message = "데이터: {data}\n{question}"

    reasoning_with_data = [
        ("system", reasoning_system_message),
        ("human", reasoning_user_message),
    ]
    reasoning_with_data_chain = ChatPromptTemplate.from_messages(reasoning_with_data) | llm | StrOutputParser()
    
    # 대답 생성
    generation = reasoning_with_data_chain.invoke({"data": data, "question": question})
    return {"question": question, "code": state['code'], "data": data, "generation": generation}

# 이전 실습에서 사용한 init_answer는 후속 코드에서 고도화 합니다
    
def answer(state: State):
    """
    데이터를 쿼리하지 않고 답변을 바로 생성합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        state (dict): LLM의 답변을 포함한 새로운 State
    """
    print("---답변 생성---") # 현재 상태를 확인하기 위한 Print문
    question = state["question"]
    
    return {"question": question, "generation": llm.invoke(question).content}   

이번 실습에서 추가할 기능에 대응하는 Node를 구현해 보겠습니다.
- 1. `plot_graph`: 그래프를 Plot하는 노드
- 2. `retrival`: RAG를 적용하는 노드
- 3. `answer_with_retrieved_data`: RAG 결과를 바탕으로 답변하는 노드

In [ ]:
def plot_graph(state: State):
    """
    현재 그래프 상태를 시각화합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        None
    """
    print("---그래프 시각화---") # 현재 상태를 확인하기 위한 Print문
    question = state["question"]

    # Draw Graph
    system_message = "당신은 주어진 데이터를 분석하는 데이터 분석가입니다.\n"
    system_message += f"주어진 DataFrame에서 데이터를 추출하여 사용자의 질문에 답할 수 있는 그래프를 그리는 코드를 작성하세요. "
    system_message += f"{df_name} DataFrame에 액세스할 수 있습니다.\n"
    system_message += f"`{df_name}` DataFrame에는 다음과 같은 열이 있습니다: {df_columns}\n"
    system_message += "데이터는 이미 로드되어 있으므로 데이터 로드 코드를 생략해야 합니다."

    message_with_data_info = [
        ("system", system_message),
        ("human", "{question}"),
    ]

    prompt_with_data_info = ChatPromptTemplate.from_messages(message_with_data_info)

    # 체인을 구성합니다.
    code_generate_chain = (
        {"question": RunnablePassthrough()}
        | prompt_with_data_info
        | llm 
        | StrOutputParser()
        | python_code_parser
    )
    code = code_generate_chain.invoke(question)
    answer = run_code(code)
    return {"question": question, "code": code, "data": answer, "generation": code}


def retrieval(state: State):
    """
    데이터 검색을 수행합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        state (dict): 검색된 데이터를 포함한 새로운 State
    """
    def get_retrieved_text(docs):
        result = "\n".join([doc.page_content for doc in docs])
        return result
    

    print("---데이터 검색---") # 현재 상태를 확인하기 위한 Print문
    question = state["question"]

    # Retrieval Chain
    retrieval_chain = (
        db_retriever
        | get_retrieved_text
    )

    data = retrieval_chain.invoke(question)

    return {"question": question, "data": data}

def answer_with_retrieved_data(state: State):
    """
    검색된 데이터를 바탕으로 답변을 생성합니다.

    Args:
        state (dict): 현재 그래프 상태

    Returns:
        state (dict): LLM의 답변을 포함한 새로운 State
    """
        # role에는 "AI 어시스턴트"가, question에는 "당신을 소개해주세요."가 들어갈 수 있습니다.

    print("---검색된 데이터를 바탕으로 답변 생성---") # 현재 상태를 확인하기 위한 Print문

    question = state["question"]
    data = state["data"]

    # 2챕터의 프롬프트와 체인을 활용합니다.
    messages_with_contexts = [
        ("system", "당신은 마케터를 위한 친절한 지원 챗봇입니다. 사용자가 입력하는 정보를 바탕으로 질문에 답하세요."),
        ("human", "정보: {context}.\n{question}."),
    ]
    prompt_with_context = ChatPromptTemplate.from_messages(messages_with_contexts)

    # 체인 구성
    qa_chain = (
        prompt_with_context
        | llm
        | StrOutputParser()
    )

    generation = qa_chain.invoke({"context": data, "question": question})
    return {"question": question, "data": data, "generation": generation}

이전 실습에서는 코드가 생성되어 ` ```python` 으로 시작한 블럭이 있다면 데이터 쿼리 분기로 넘기고, 그렇지 않다면 단순 답변 분기로 넘기는 로직을 사용했습니다. <br>
이를 더 고도화 하여, **라우팅 전문가 페르소나**를 적용한 체인을 통해 원하는 로직을 선택할 수 있도록 설정해봅시다. 

라우터란?
라우터는 네트워크의 교통정리를 하는 장치예요. 
집, 회사 또는 인터넷 환경에서 여러 기기(컴퓨터, 스마트폰 등)를 인터넷에 연결해주고, 
데이터가 어디로 가야 할지 알려주는 역할을 합니다.

In [ ]:
# 시스템 메시지에 사용 가능한 툴과 각 툴을 사용할 상황을 명시합니다.
# 수월한 선택을 위해 JSON 형식으로 출력하도록 프롬프트에 지정합니다.
route_system_message = """당신은 사용자의 질문에 RAG, 엑셀 데이터 중 어떤 것을 활용할 수 있는지 결정하는 전문가입니다. \n
시장 조사와 관련된 질문이라면 RAG를 활용하세요. \n
잉크젯 데이터와 관련된 질문이라면 excel_data를 활용하세요. \n
그래프를 그리라는 질문이면 excel_plot을 활용하세요. \n
둘 다 아니라면, plain_answer로 충분합니다. \n
주어진 질문에 맞춰 `rag`, `excel_data`, `excel_plot`, `plain_answer`중 하나를 선택하세요. \n
답변은 `route` key 하나만 있는 JSON으로 답변하고, 다른 텍스트나 설명을 생성하지 마세요."""
route_user_message = "{question}"
route_prompt = ChatPromptTemplate.from_messages([
    ("system", route_system_message),
    ("human", route_user_message)   
])

# 로직 선택용 ChatOllama 객체를 생성합니다.
# 출력 양식을 json으로 명시하고, 같은 질문에 같은 로직을 적용하기 위해 temperature를 0으로 설정합니다.
route_llm = ChatOllama(model="mistral:7b", format="json", temperature=0)
router_chain = route_prompt | route_llm | JsonOutputParser()

다양한 질문에 대해 테스트 해보고, 그 결과를 확인해 봅시다.

In [ ]:
print(router_chain.invoke({"question": "올해 키오스크 시장의 전망을 알려줘"})) # 2챕터에서 사용한 RAG 주제
print(router_chain.invoke({"question": "잉크젯 데이터의 통계 그래프를 그려줘"})) # 3~4챕터에서 사용한 엑셀 데이터 - 그래프 주제
print(router_chain.invoke({"question": "오늘 저녁 뭐 먹을까?"})) # 일반 질문
print(router_chain.invoke({"question": "잉크젯 데이터의 각 컬럼의 평균값을 알려줘"})) # 3챕터에서 사용한 엑셀 데이터 - 데이터 쿼리 주제

대부분 잘 결정하는 것을 볼 수 있습니다.<br>
이 체인을 활용해서 고도화된 `init_answer` 함수를 정의합니다.

In [ ]:
def init_answer(state: State) -> str:
    """
    초기 질문의 경로를 결정합니다.
    주어진 상태(State) 객체에서 질문을 추출하고, 해당 질문을 기반으로 
    router_chain을 호출하여 경로(route)를 생성합니다.
    경로 정보는 나중에 활용하기 위해 반환 객체에 포함됩니다.

    Parameters:
    state (State): 상태 정보를 담고 있는 객체로, "question" 키를 포함합니다.

    Returns:
    dict: 질문과 생성된 경로 정보를 포함하는 딕셔너리입니다.
    """
    # state에서 "question" 키를 통해 질문 내용을 추출합니다.
    question = state["question"]
    
    # 추출한 질문을 router_chain으로 전달하여 해당 질문에 대한 경로를 생성합니다.
    # router_chain의 invoke 메소드는 {"question": 질문} 형태로 입력을 받고
    # 반환 값에서 "route" 키로 생성된 경로를 반환합니다.
    route = router_chain.invoke({"question": question})["route"]
    
    # 질문과 생성된 경로 정보를 포함하는 딕셔너리를 반환합니다.
    return {"question": question, "generation": route}

def route_question(state: State) -> str:
    """
    주어진 상태(State) 객체에서 경로 정보를 추출하고, 이를 소문자와
    공백 제거 처리를 한 후 반환합니다. 이 함수는 특정한 경로 정보를
    클린하게 가공하는 역할을 합니다.

    Parameters:
    state (State): 상태 정보를 담고 있는 객체로, "generation" 키를 포함합니다.

    Returns:
    str: 경로 정보를 소문자로 변환하고, 앞뒤 공백을 제거한 문자열입니다.
    """
    # state에서 "generation" 키로 저장된 경로 정보를 가져옵니다.
    route = state["generation"]
    
    # 경로 정보를 소문자로 변환하고 앞뒤 공백을 제거하여 반환합니다.
    return route.lower().strip()


지난 실습과 달리 총 4개의 노드를 사용하고, 조건에 따라 사용하는 노드가 달라집니다.

이를 구현하기 위해, 노드와 간선을 그래프에 추가합니다.

In [ ]:
## 그래프 구성

# 앞서 정의한 Node를 모두 추가합니다.
workflow.add_node("init_answer", init_answer)

workflow.add_node("excel_data", query)
workflow.add_node("rag", retrieval)

workflow.add_node("excel_plot", plot_graph)
workflow.add_node("answer_with_data", answer_with_data)
workflow.add_node("plain_answer", answer)
workflow.add_node("answer_with_retrieval", answer_with_retrieved_data)

# 시작지점을 정의합니다.
workflow.set_entry_point("init_answer")

# 간선을 정의합니다.
# END는 종결 지점을 의미합니다.
workflow.add_edge("plain_answer", END) # workflow.set_finish_point("answer")와 동일합니다.
workflow.add_edge("answer_with_data", END)
workflow.add_edge("answer_with_retrieval", END)
workflow.add_edge("excel_plot", END) # 그래프를 그리고 종결합니다.
workflow.add_edge("excel_data", "answer_with_data")
workflow.add_edge("rag", "answer_with_retrieval")


# 조건부 간선을 정의합니다.
# init_answer 노드의 답변을 바탕으로 decide_query 함수에서 query 또는 answer로 분기합니다.
workflow.add_conditional_edges(
    "init_answer",
    route_question,
    # 어떤 노드로 이동할지 mapping합니다. 없어도 무방하지만, Graph의 가독성을 높일 수 있습니다.
    {
        "excel_data": "excel_data",
        "rag": "rag",
        "excel_plot": "excel_plot",
        "plain_answer": "plain_answer"
    }
)

Node, Edge, 분기를 모두 구성했으니 이제 그래프를 컴파일 하고, 그 구조를 확인해 봅시다.

In [ ]:
graph = workflow.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

이제 챗봇을 사용해봅시다.<br>
다양한 종류의 데이터가 로드된 만큼, 어떤 데이터에 대한 요청인지 프롬프트에 명시하는 것이 좋습니다.

- 예시 질문 (시장 조사 문서 활용): 올해 키오스크 시장의 전망을 알려줘
- 예시 질문 (잉크젯 데이터 활용): 잉크젯 데이터의 각 컬럼의 평균값을 알려줘
- 예시 질문 (잉크젯 그래프): 잉크젯 데이터의 통계 그래프를 그려줘
- 예시 질문 (데이터 무관): 오늘 저녁 뭐 먹을까?

In [ ]:
while True:
    question = input("질문을 입력해주세요 (종료를 원하시면 '종료'를 입력해주세요.): ")
    if question == "종료":
        break
    else:
        # graph.invoke 함수를 사용하여 그래프를 실행하고, 최종 결과를 반환합니다.
        # 답변 생성에는 약 1분 정도 소요됩니다.
        print("Assistant: ", graph.invoke({"question": ("user", question)})['generation'])

### 추가 실습
- 챗봇이 그린 그래프를 화면에 출력하는 기능을 추가하고자 합니다. 이를 위해 챗봇이 그린 그래프를 항상 `plot.png` 파일로 저장하고자 합니다.
- 다음 챕터에서 이 기능을 포함한 Streamlit 기반 챗봇을 구현할 것입니다. 그 전에, 먼저 이 기능을 구현해봅시다.
  - Hint. 프롬프트 엔지니어링을 통해 그래프를 그리는 코드를 반드시 `plt.plot()`으로 끝나게 하고, 그 코드를 그래프를 저장하는 코드로 전처리 해보세요.